### Setup

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.parent.absolute()))
import config
import argparse
from pprint import pprint
from time import sleep
from context import Context
from contracts.farm_contract import FarmContract
from contracts.staking_contract import StakingContract
from contracts.metastaking_contract import MetaStakingContract
from contracts.unstaker_contract import UnstakerContract
from contracts.router_contract import RouterContract
from contracts.permissions_hub_contract import PermissionsHubContract
from contracts.pair_contract import PairContract
from contracts.fees_collector_contract import FeesCollectorContract
from utils.utils_chain import base64_to_hex, WrapperAddress
from tools.runners.farm_runner import upgrade_farmv2_contract
from tools.runners.staking_runner import upgrade_staking_contracts
from tools.runners.metastaking_runner import upgrade_metastaking_contract
from tools.runners.generic_runner import upgrade_generic_contract

context = Context()

In [ ]:
farm_contract = FarmContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqr9gx79xls33tnzt0a94uvnkf452fz26q2jpsau4ulw")
staking_contract = StakingContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqqsx29p3fge7upkgup4mm5xsdsv4w7rh82jpsdcvdrt")
metastaking_contract = MetaStakingContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqxjdlk9d8gap8q2ghey6q5cg69y7p4rkd2jpsv89u3n")
unstaker_contract: UnstakerContract = context.get_contracts(config.UNSTAKERS)[0]
router_contract: RouterContract = context.get_contracts(config.ROUTER_V2)[0]
fees_collector_contract: FeesCollectorContract = context.get_contracts(config.FEES_COLLECTORS)[0]

print("Farm contract:")
pprint(farm_contract.get_config_dict())
print("Staking contract:")
pprint(staking_contract.get_config_dict())
print("MetaStaking contract:")
pprint(metastaking_contract.get_config_dict())
print("Unstaker contract:")
pprint(unstaker_contract.get_config_dict())
print("Router contract:")
pprint(router_contract.get_config_dict())
print("Fees collector contract:")
pprint(fees_collector_contract.get_config_dict())

### Farm, staking, metastaking v3.2 upgrades and setup

Deploy permissions hub

In [ ]:
permissions_hub_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/permissions-hub.wasm"
expected_code_hash = "5ae2ac86ff5368649735a5c890357c9a7980775820943b721cf3d080f20de262"

permissions_hub_contract = PermissionsHubContract("")
tx_hash, permissions_hub_contract.address = permissions_hub_contract.contract_deploy(context.deployer_account, context.network_provider.proxy, 
                                                                                     permissions_hub_path, [])

if not context.network_provider.check_complex_tx_status(tx_hash, f"deploy permissions hub"):
    raise Exception("Failed to deploy permissions hub")

code_hash = context.network_provider.proxy.get_account(WrapperAddress(permissions_hub_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Farm pause

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = farm_contract.pause(context.deployer_account, context.network_provider.proxy)

Staking pause

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = staking_contract.pause(context.deployer_account, context.network_provider.proxy)

Farm upgrade

In [ ]:
farm_bytecode_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/farm-with-locked-rewards.wasm"
expected_code_hash = "c1f2a3652371510767f5d64a4df02a3eda2744d54e9d91f7680dbf1bd02d82bb"

args = argparse.Namespace(address=farm_contract.address, bytecode=farm_bytecode_path, compare_states=True, all=False)
upgrade_farmv2_contract(args)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(farm_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Staking upgrade

In [ ]:
staking_bytecode_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/farm-staking.wasm"
expected_code_hash = "7eddcb67bef25c4a0c7de0c3e1b2444cbf84e05546ac61c29726b568f9efffde"

args = argparse.Namespace(address=staking_contract.address, bytecode=staking_bytecode_path, compare_states=True, all=False)
upgrade_staking_contracts(args)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(staking_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Metastaking upgrade

In [ ]:
metastaking_bytecode_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/farm-staking-proxy.wasm"
expected_code_hash = "8a605fa8539f193a49d267ce73f9010904796d6b930ea7e4e09f9b4d9da1af9f"

args = argparse.Namespace(address=metastaking_contract.address, bytecode=metastaking_bytecode_path, compare_states=True, all=False)
upgrade_metastaking_contract(args)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(metastaking_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Set permissions hub in farm, staking and metastaking

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
tx_hash = staking_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
tx_hash = metastaking_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)

Farm resume

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = farm_contract.resume(context.deployer_account, context.network_provider.proxy)

Staking resume

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = staking_contract.resume(context.deployer_account, context.network_provider.proxy)

### Unstaker upgrade

Unstaker upgrade

In [ ]:
unstaker_bytecode_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/token-unstake.wasm"
expected_code_hash = "b22c402e39ba46d6b0521d51e99e51c277b7cbf87c3795953754c084320c6300"

args = argparse.Namespace(address=unstaker_contract.address, bytecode=unstaker_bytecode_path, compare_states=True, all=False)
upgrade_generic_contract(args)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(unstaker_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Unstaker burn fees reduction

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = unstaker_contract.set_fees_burn_percentage(context.deployer_account, context.network_provider.proxy, [0])

### New common LP token

In [ ]:
tx = router_contract.add_common_tokens_for_user_pairs(context.deployer_account, context.network_provider.proxy, "USH-111e09")

In [ ]:

tx = router_contract.config_enable_by_user_parameters(context.deployer_account, context.network_provider.proxy, common_token_id="USH-111e09", locked_token_id="LKESDT-fc19ba", 
                                                      min_locked_token_value=900000000000000000, min_lock_period_epochs=2)

Fees collector setup

In [ ]:
fees_collector_contract.add_known_tokens(context.deployer_account, context.network_provider.proxy, ["USH-111e09"])

### Buyback and burn + fees collector setup

In [ ]:
pair_addresses = ["erd1qqqqqqqqqqqqqpgqs8r2jhfymgle49dqx42xyypx6r2smt602jps2kcn8f"]

context.deployer_account.sync_nonce(context.network_provider.proxy)
mex_contract = PairContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2") # egldmex contract

for pair_address in pair_addresses:
    pair_contract = PairContract.load_contract_by_address(pair_address)    # operating pair

    print(f"Setting up pair {pair_contract.lpToken}: {pair_address}")

    # whitelist in egldmex pair for swap no fees
    mex_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, pair_contract.address)

    # set where to swap and what to do with the fees
    pair_contract.add_trusted_swap_pair(context.deployer_account, context.network_provider.proxy,
                                        [
                                            mex_contract.address,
                                            mex_contract.firstToken,
                                            mex_contract.secondToken
                                        ])
    
    pair_contract.set_fees_percents(context.deployer_account, context.network_provider.proxy,
                                [300, 100])
    
    pair_contract.set_fee_on_via_router(context.deployer_account, context.network_provider.proxy, router_contract, 
                                [
                                    config.ZERO_CONTRACT_ADDRESS,
                                    mex_contract.secondToken
                                ])
    
    fees_collector_contract.add_known_contracts(context.deployer_account, context.network_provider.proxy,
                                                [
                                                    pair_contract.address
                                                ])
    pair_contract.add_fees_collector(context.deployer_account, context.network_provider.proxy,
                                    [fees_collector_contract.address, 50000])
    
    input("Press Enter to continue...")